# 🤖 AI SEO Architects - Финальная версия нейро-сотрудника

**Полная демонстрация 14 агентов с ChromaDB RAG системой и визуализациями**

---

### 📋 Инструкция для пользователя:

1. **Выполните все ячейки 1-12** по порядку для определения функций
2. **Запустите ячейку 13 (ТЕСТОВАЯ ЯЧЕЙКА)** - введите свои данные
3. **Получите результаты тестирования** всех 14 агентов с визуализациями

### 🎯 Что будет протестировано:
- ✅ **Executive уровень** (2 агента): Chief SEO Strategist, Business Development Director
- ✅ **Management уровень** (4 агента): Task Coordination, Sales Operations, Technical SEO Operations, Client Success
- ✅ **Operational уровень** (8 агентов): Lead Qualification, Sales Conversation, Proposal Generation, Technical SEO Auditor, Content Strategy, Link Building, Competitive Analysis, Reporting
- ✅ **Пайплайн сценарии**: Lead→Sales→Proposal, SEO Audit→Content→Link Building
- ✅ **Визуализации**: Графики производительности, диаграммы успешности, dashboard

---

In [ ]:
# ЯЧЕЙКА 1: Функция установки окружения

def setup_environment(openai_api_key=None, project_path='/content/ai-seo-architects', github_repo='https://github.com/Andrew821667/ai-seo-architects.git'):
    """
    Настройка окружения для AI SEO Architects с ChromaDB
    
    Args:
        openai_api_key (str, optional): OpenAI API ключ
        project_path (str): Путь к проекту (по умолчанию для Colab)
        github_repo (str): URL GitHub репозитория
        
    Returns:
        dict: Результат настройки окружения
    """
    import os
    import sys
    import subprocess
    import asyncio
    import nest_asyncio
    
    try:
        print('🔧 Настройка окружения AI SEO Architects')
        print('=' * 50)
        
        # Клонируем проект если нужно
        if not os.path.exists(project_path):
            print(f'📥 Клонирование проекта из {github_repo}')
            subprocess.run(['git', 'clone', github_repo, project_path], check=True)
            print('✅ Проект клонирован')
        else:
            print('✅ Проект уже существует')
            
        # Переходим в директорию проекта
        os.chdir(project_path)
        if project_path not in sys.path:
            sys.path.append(project_path)
            
        # Устанавливаем зависимости с фиксированными версиями
        dependencies = [
            'openai==1.6.1',
            'langchain==0.1.7', 
            'langchain-openai==0.0.8',
            'langgraph==0.0.45',
            'chromadb==0.4.15',
            'numpy==1.25.2',
            'nest-asyncio==1.5.8',
            'matplotlib>=3.7.0',
            'plotly>=5.15.0',
            'seaborn>=0.12.0',
            'pandas>=2.0.0'
        ]
        
        print('\n📦 Установка зависимостей:')
        for dep in dependencies:
            try:
                subprocess.run(['pip', 'install', '-q', dep], check=True)
                print(f'✅ {dep}')
            except:
                print(f'⚠️ {dep} (ошибка установки, но продолжаем)')
                
        # Настройка asyncio для Jupyter
        nest_asyncio.apply()
        
        # Настройка OpenAI API ключа
        if openai_api_key:
            os.environ['OPENAI_API_KEY'] = openai_api_key
            print('✅ OpenAI API ключ установлен через параметр')
        else:
            # Пробуем загрузить из Colab secrets
            try:
                from google.colab import userdata
                api_key = userdata.get('OPENAI_API_KEY')
                if api_key:
                    os.environ['OPENAI_API_KEY'] = api_key
                    print('✅ OpenAI API ключ загружен из Colab secrets')
                else:
                    print('⚠️ OpenAI API ключ не найден в secrets - будет запрошен в тестовой ячейке')
            except:
                print('⚠️ OpenAI API ключ будет запрошен в тестовой ячейке')
                
        print('\n🎯 Окружение успешно настроено!')
        
        return {
            'success': True,
            'project_path': project_path,
            'dependencies_installed': len(dependencies),
            'openai_configured': 'OPENAI_API_KEY' in os.environ
        }
        
    except Exception as e:
        print(f'❌ Ошибка настройки окружения: {e}')
        return {
            'success': False,
            'error': str(e)
        }

In [ ]:
# ЯЧЕЙКА 2: Функция инициализации ChromaDB системы

def initialize_chromadb_system(knowledge_path='./knowledge/', vector_store_path='./data/vector_stores/'):
    """
    Инициализация ChromaDB системы знаний для всех агентов
    
    Args:
        knowledge_path (str): Путь к базам знаний
        vector_store_path (str): Путь к векторным хранилищам
        
    Returns:
        dict: Результат инициализации ChromaDB системы
    """
    try:
        import os
        from knowledge.chroma_knowledge_manager import knowledge_manager
        from core.config import config
        
        print('🔧 Инициализация ChromaDB Knowledge Manager')
        print('=' * 50)
        print(f'📁 База знаний: {config.KNOWLEDGE_BASE_PATH}')
        print(f'🔍 RAG включен: {config.ENABLE_RAG}')
        print(f'📊 Chunk size: {config.RAG_CHUNK_SIZE}')
        print(f'🎯 Top K: {config.RAG_TOP_K}')
        print(f'🗄️ Векторная БД: ChromaDB (замена FAISS)')
        print(f'💾 Путь хранения: {config.VECTOR_STORE_PATH}')
        
        # Проверяем структуру директорий знаний
        knowledge_stats = {}
        total_knowledge_files = 0
        
        for level in ['executive', 'management', 'operational']:
            level_path = os.path.join(config.KNOWLEDGE_BASE_PATH, level)
            if os.path.exists(level_path):
                md_files = [f for f in os.listdir(level_path) if f.endswith('.md')]
                knowledge_stats[level] = len(md_files)
                total_knowledge_files += len(md_files)
                print(f'📚 {level:12}: {len(md_files):2d} файлов знаний')
            else:
                knowledge_stats[level] = 0
                print(f'❌ {level:12}: директория не найдена')
                
        print(f'\n📄 Общее количество файлов знаний: {total_knowledge_files}')
        
        # Инициализируем базы знаний для всех 14 агентов в ChromaDB
        print('\n🚀 Инициализация всех баз знаний в ChromaDB...')
        print('=' * 50)
        initialization_results = knowledge_manager.initialize_all_agents_knowledge()
        
        successful_agents = [agent for agent, success in initialization_results.items() if success]
        failed_agents = [agent for agent, success in initialization_results.items() if not success]
        
        print(f'\n✅ Успешно в ChromaDB: {len(successful_agents)}/14 агентов')
        print(f'❌ Ошибки: {len(failed_agents)} агентов')
        
        if successful_agents:
            print(f'\n✅ Успешные агенты:')
            for i, agent in enumerate(successful_agents, 1):
                print(f'   {i:2d}. {agent}')
                
        if failed_agents:
            print(f'\n❌ Проблемные агенты: {", ".join(failed_agents)}')
            
        # Получаем статистику ChromaDB
        chroma_stats = knowledge_manager.get_stats()
        total_docs = sum(store.get('documents_count', 0) for store in chroma_stats['stores'].values())
        
        print(f'\n📊 СТАТИСТИКА CHROMADB:')
        print('=' * 30)
        print(f'💾 Коллекций ChromaDB: {chroma_stats["vector_stores_count"]}')
        print(f'📄 Документов всего: {total_docs:,}')
        print(f'🔗 OpenAI Embeddings: {"✅ Активны" if chroma_stats["embeddings_available"] else "❌ Недоступны"}')
        
        # Детальная статистика по агентам
        print(f'\n📋 ДЕТАЛЬНАЯ СТАТИСТИКА ПО АГЕНТАМ:')
        for agent_name, store_info in chroma_stats['stores'].items():
            status = store_info.get('status', 'unknown')
            doc_count = store_info.get('documents_count', 0)
            print(f'   {agent_name:30} | {doc_count:3d} docs | {status}')
            
        success_rate = len(successful_agents) / 14 * 100
        print(f'\n🎯 ChromaDB готова: {success_rate:.1f}% агентов инициализированы')
        
        return {
            'success': True,
            'knowledge_manager': knowledge_manager,
            'successful_agents': successful_agents,
            'failed_agents': failed_agents,
            'initialization_results': initialization_results,
            'chroma_stats': chroma_stats,
            'total_documents': total_docs,
            'success_rate': success_rate,
            'knowledge_stats': knowledge_stats
        }
        
    except Exception as e:
        print(f'❌ Ошибка инициализации ChromaDB: {e}')
        return {
            'success': False,
            'error': str(e)
        }

In [ ]:
# ЯЧЕЙКА 3: Функция создания всех 14 агентов

def create_all_agents(chromadb_system_result, enable_rag=True, mock_mode=False):
    """
    Создание всех 14 агентов с ChromaDB RAG поддержкой
    
    Args:
        chromadb_system_result (dict): Результат инициализации ChromaDB
        enable_rag (bool): Включить RAG поддержку
        mock_mode (bool): Режим без OpenAI API (для тестирования)
        
    Returns:
        dict: Словарь всех созданных агентов по уровням
    """
    try:
        print('🤖 СОЗДАНИЕ ВСЕХ 14 АГЕНТОВ С CHROMADB RAG')
        print('=' * 60)
        
        if not chromadb_system_result['success']:
            print('❌ ChromaDB система не инициализирована')
            return {'success': False, 'error': 'ChromaDB not initialized'}
        
        successful_agents = chromadb_system_result['successful_agents']
        agents = {
            'executive': {},
            'management': {},
            'operational': {}
        }
        
        # Конфигурация всех 14 агентов
        agents_config = {
            'executive': [
                ('chief_seo_strategist', 'agents.executive.chief_seo_strategist', 'ChiefSEOStrategistAgent'),
                ('business_development_director', 'agents.executive.business_development_director', 'BusinessDevelopmentDirectorAgent')
            ],
            'management': [
                ('task_coordination', 'agents.management.task_coordination', 'TaskCoordinationAgent'),
                ('sales_operations_manager', 'agents.management.sales_operations_manager', 'SalesOperationsManagerAgent'),
                ('technical_seo_operations_manager', 'agents.management.technical_seo_operations_manager', 'TechnicalSEOOperationsManagerAgent'),
                ('client_success_manager', 'agents.management.client_success_manager', 'ClientSuccessManagerAgent')
            ],
            'operational': [
                ('lead_qualification', 'agents.operational.lead_qualification', 'LeadQualificationAgent'),
                ('sales_conversation', 'agents.operational.sales_conversation', 'SalesConversationAgent'),
                ('proposal_generation', 'agents.operational.proposal_generation', 'ProposalGenerationAgent'),
                ('technical_seo_auditor', 'agents.operational.technical_seo_auditor', 'TechnicalSEOAuditorAgent'),
                ('content_strategy', 'agents.operational.content_strategy', 'ContentStrategyAgent'),
                ('link_building', 'agents.operational.link_building', 'LinkBuildingAgent'),
                ('competitive_analysis', 'agents.operational.competitive_analysis', 'CompetitiveAnalysisAgent'),
                ('reporting', 'agents.operational.reporting', 'ReportingAgent')
            ]
        }
        
        creation_stats = {
            'executive': {'created': 0, 'failed': 0},
            'management': {'created': 0, 'failed': 0},
            'operational': {'created': 0, 'failed': 0}
        }
        
        # Создаем агентов по уровням
        for level, level_agents in agents_config.items():
            print(f'\n📊 {level.upper()} LEVEL ({len(level_agents)} агентов):')
            print('-' * 40)
            
            for agent_id, module_path, class_name in level_agents:
                try:
                    # Проверяем что база знаний готова
                    if agent_id in successful_agents or mock_mode:
                        # Динамический импорт класса агента
                        module = __import__(module_path, fromlist=[class_name])
                        AgentClass = getattr(module, class_name)
                        
                        # Создаем агента с правильными параметрами
                        agent = AgentClass(
                            agent_id=agent_id,
                            name=agent_id.replace('_', ' ').title(),
                            agent_level=level,
                            rag_enabled=enable_rag and not mock_mode,
                            knowledge_base=f'{level}/{agent_id}.md'
                        )
                        
                        agents[level][agent_id] = agent
                        creation_stats[level]['created'] += 1
                        
                        rag_status = '✅' if (enable_rag and not mock_mode and agent_id in successful_agents) else '⚠️'
                        mode = 'MOCK' if mock_mode else 'RAG'
                        print(f'✅ {agent_id:30} | {level:12} | {mode} {rag_status}')
                        
                    else:
                        creation_stats[level]['failed'] += 1
                        print(f'⚠️ {agent_id:30} | База знаний не готова в ChromaDB')
                        
                except Exception as e:
                    creation_stats[level]['failed'] += 1
                    print(f'❌ {agent_id:30} | Ошибка: {str(e)[:40]}...')
        
        # Итоговая статистика создания агентов
        total_created = sum(stats['created'] for stats in creation_stats.values())
        total_failed = sum(stats['failed'] for stats in creation_stats.values())
        
        print(f'\n📊 ИТОГОВАЯ СТАТИСТИКА СОЗДАНИЯ АГЕНТОВ:')
        print('=' * 50)
        for level, stats in creation_stats.items():
            print(f'{level:15}: {stats["created"]:2d} создано, {stats["failed"]:2d} ошибок')
        print(f'\n🎯 ОБЩИЙ РЕЗУЛЬТАТ: {total_created}/14 агентов создано ({total_created/14*100:.1f}%)')
        
        return {
            'success': True,
            'agents': agents,
            'creation_stats': creation_stats,
            'total_created': total_created,
            'total_failed': total_failed,
            'success_rate': total_created/14*100
        }
        
    except Exception as e:
        print(f'❌ Ошибка создания агентов: {e}')
        return {
            'success': False,
            'error': str(e)
        }

In [ ]:
# ЯЧЕЙКА 4: Функция тестирования Executive агентов с визуализацией

async def test_executive_agents(agents_result, test_scenarios=None):
    """
    Тестирование Executive уровня агентов с визуализацией
    
    Args:
        agents_result (dict): Результат создания агентов
        test_scenarios (list): Пользовательские тестовые сценарии
        
    Returns:
        dict: Результаты тестирования Executive агентов
    """
    try:
        import time
        import asyncio
        
        print('👑 ТЕСТИРОВАНИЕ EXECUTIVE АГЕНТОВ')
        print('=' * 60)
        
        if not agents_result['success']:
            return {'success': False, 'error': 'Agents not created'}
            
        executive_agents = agents_result['agents']['executive']
        
        # Тестовые сценарии для Executive уровня
        default_scenarios = [
            {
                'agent_id': 'chief_seo_strategist',
                'scenario': 'Enterprise SEO стратегия',
                'task_data': {
                    'client_type': 'Enterprise',
                    'budget': 15000000,  # 15M ₽/год
                    'industry': 'E-commerce',
                    'current_traffic': 2500000,  # 2.5M посещений
                    'target_growth': 0.40,  # 40% рост
                    'timeframe': '12 месяцев'
                },
                'expected_metrics': ['strategic_recommendations', 'kpi_targets', 'implementation_roadmap']
            },
            {
                'agent_id': 'business_development_director',
                'scenario': 'Enterprise сделка закрытие',
                'task_data': {
                    'deal_size': 25000000,  # 25M ₽
                    'client_industry': 'Technology',
                    'decision_makers': 3,
                    'competition_level': 'high',
                    'timeline': '6 months',
                    'current_stage': 'proposal_review'
                },
                'expected_metrics': ['win_probability', 'risk_assessment', 'next_steps']
            }
        ]
        
        scenarios = test_scenarios or default_scenarios
        results = []
        
        for scenario in scenarios:
            agent_id = scenario['agent_id']
            
            if agent_id not in executive_agents:
                print(f'⚠️ Агент {agent_id} не создан, пропускаем')
                continue
                
            print(f'\n🧪 ТЕСТИРОВАНИЕ: {scenario["scenario"]}')
            print(f'🤖 Агент: {agent_id}')
            print('-' * 50)
            
            agent = executive_agents[agent_id]
            
            try:
                start_time = time.time()
                
                # Выполняем задачу агента
                task_result = await agent.process_task_with_retry(scenario['task_data'])
                
                processing_time = time.time() - start_time
                
                # Анализируем результат
                success = task_result.get('success', False)
                
                if success:
                    print(f'✅ Тест пройден за {processing_time:.2f}с')
                    
                    # Извлекаем метрики из результата
                    result_content = task_result.get('result', '')
                    
                    # Подсчет качественных метрик
                    quality_score = min(100, len(result_content) / 10)  # Базовая оценка качества
                    
                    # Проверка наличия ожидаемых метрик
                    expected_metrics = scenario.get('expected_metrics', [])
                    metrics_found = sum(1 for metric in expected_metrics if metric.lower() in result_content.lower())
                    metrics_coverage = metrics_found / max(1, len(expected_metrics)) * 100
                    
                    print(f'📊 Качество ответа: {quality_score:.1f}/100')
                    print(f'🎯 Покрытие метрик: {metrics_coverage:.1f}% ({metrics_found}/{len(expected_metrics)})')
                    
                    # Показываем превью результата
                    if result_content:
                        preview = result_content[:200].replace('\n', ' ')
                        print(f'📄 Превью результата: {preview}...')
                else:
                    print(f'❌ Тест провален: {task_result.get("error", "Unknown error")}')
                    quality_score = 0
                    metrics_coverage = 0
                    
                # Сохраняем результат для визуализации
                results.append({
                    'agent_id': agent_id,
                    'scenario': scenario['scenario'],
                    'success': success,
                    'processing_time': processing_time,
                    'quality_score': quality_score,
                    'metrics_coverage': metrics_coverage,
                    'task_result': task_result
                })
                
            except Exception as e:
                print(f'❌ Ошибка тестирования {agent_id}: {str(e)[:60]}...')
                results.append({
                    'agent_id': agent_id,
                    'scenario': scenario['scenario'],
                    'success': False,
                    'processing_time': 0,
                    'quality_score': 0,
                    'metrics_coverage': 0,
                    'error': str(e)
                })
                
        # Итоговая статистика Executive уровня
        successful_tests = len([r for r in results if r['success']])
        avg_processing_time = sum(r['processing_time'] for r in results if r['success']) / max(1, successful_tests)
        avg_quality_score = sum(r['quality_score'] for r in results if r['success']) / max(1, successful_tests)
        avg_metrics_coverage = sum(r['metrics_coverage'] for r in results if r['success']) / max(1, successful_tests)
        
        print(f'\n📊 СТАТИСТИКА EXECUTIVE УРОВНЯ:')
        print('=' * 40)
        print(f'✅ Успешных тестов: {successful_tests}/{len(results)}')
        print(f'⏱️ Среднее время: {avg_processing_time:.2f}с')
        print(f'🎯 Среднее качество: {avg_quality_score:.1f}/100')
        print(f'📈 Покрытие метрик: {avg_metrics_coverage:.1f}%')
        
        return {
            'success': True,
            'level': 'executive',
            'results': results,
            'stats': {
                'successful_tests': successful_tests,
                'total_tests': len(results),
                'success_rate': successful_tests / max(1, len(results)) * 100,
                'avg_processing_time': avg_processing_time,
                'avg_quality_score': avg_quality_score,
                'avg_metrics_coverage': avg_metrics_coverage
            }
        }
        
    except Exception as e:
        print(f'❌ Ошибка тестирования Executive агентов: {e}')
        return {
            'success': False,
            'error': str(e)
        }

In [ ]:
# ЯЧЕЙКА 5: Функция тестирования Management агентов с визуализацией

async def test_management_agents(agents_result, test_scenarios=None):
    """
    Тестирование Management уровня агентов с визуализацией
    
    Args:
        agents_result (dict): Результат создания агентов
        test_scenarios (list): Пользовательские тестовые сценарии
        
    Returns:
        dict: Результаты тестирования Management агентов
    """
    try:
        import time
        import asyncio
        
        print('🎯 ТЕСТИРОВАНИЕ MANAGEMENT АГЕНТОВ')
        print('=' * 60)
        
        if not agents_result['success']:
            return {'success': False, 'error': 'Agents not created'}
            
        management_agents = agents_result['agents']['management']
        
        # Тестовые сценарии для Management уровня
        default_scenarios = [
            {
                'agent_id': 'task_coordination',
                'scenario': 'Координация комплексного проекта',
                'task_data': {
                    'project_type': 'Enterprise SEO Audit',
                    'tasks_count': 15,
                    'agents_required': ['technical_seo_auditor', 'content_strategy', 'competitive_analysis'],
                    'deadline': '2 weeks',
                    'priority': 'high',
                    'client_value': 5000000
                },
                'expected_metrics': ['task_assignment', 'timeline', 'resource_allocation']
            },
            {
                'agent_id': 'sales_operations_manager',
                'scenario': 'Анализ sales pipeline',
                'task_data': {
                    'total_leads': 150,
                    'qualified_leads': 45,
                    'proposals_sent': 28,
                    'deals_won': 8,
                    'revenue_target': 50000000,  # 50M ₽
                    'period': 'Q4 2024'
                },
                'expected_metrics': ['conversion_rates', 'pipeline_health', 'forecasting']
            },
            {
                'agent_id': 'technical_seo_operations_manager',
                'scenario': 'Мониторинг технических показателей',
                'task_data': {
                    'sites_monitored': 25,
                    'critical_issues': 8,
                    'core_web_vitals_failed': 12,
                    'crawling_errors': 156,
                    'avg_response_time': 3.2,
                    'period': 'current_month'
                },
                'expected_metrics': ['issue_prioritization', 'performance_trends', 'action_plan']
            },
            {
                'agent_id': 'client_success_manager',
                'scenario': 'Анализ клиентского успеха',
                'task_data': {
                    'client_portfolio': 35,
                    'at_risk_clients': 4,
                    'upsell_opportunities': 8,
                    'nps_score': 72,
                    'churn_rate': 0.08,
                    'expansion_revenue': 12000000  # 12M ₽
                },
                'expected_metrics': ['churn_prediction', 'upsell_matrix', 'retention_strategy']
            }
        ]
        
        scenarios = test_scenarios or default_scenarios
        results = []
        
        for scenario in scenarios:
            agent_id = scenario['agent_id']
            
            if agent_id not in management_agents:
                print(f'⚠️ Агент {agent_id} не создан, пропускаем')
                continue
                
            print(f'\n🧪 ТЕСТИРОВАНИЕ: {scenario["scenario"]}')
            print(f'🤖 Агент: {agent_id}')
            print('-' * 50)
            
            agent = management_agents[agent_id]
            
            try:
                start_time = time.time()
                
                # Выполняем задачу агента
                task_result = await agent.process_task_with_retry(scenario['task_data'])
                
                processing_time = time.time() - start_time
                success = task_result.get('success', False)
                
                if success:
                    print(f'✅ Тест пройден за {processing_time:.2f}с')
                    
                    result_content = task_result.get('result', '')
                    quality_score = min(100, len(result_content) / 8)  # Management качество
                    
                    expected_metrics = scenario.get('expected_metrics', [])
                    metrics_found = sum(1 for metric in expected_metrics if metric.lower() in result_content.lower())
                    metrics_coverage = metrics_found / max(1, len(expected_metrics)) * 100
                    
                    # Management специфичные метрики
                    efficiency_score = min(100, 50 + (30 - processing_time) * 2)  # Быстрота выполнения
                    actionability_score = min(100, metrics_coverage * 1.2)  # Практичность рекомендаций
                    
                    print(f'📊 Качество: {quality_score:.1f}/100')
                    print(f'⚡ Эффективность: {efficiency_score:.1f}/100')
                    print(f'🎯 Практичность: {actionability_score:.1f}/100')
                    print(f'📈 Покрытие метрик: {metrics_coverage:.1f}%')
                    
                    if result_content:
                        preview = result_content[:180].replace('\n', ' ')
                        print(f'📄 Превью: {preview}...')
                        
                else:
                    print(f'❌ Тест провален: {task_result.get("error", "Unknown error")}')
                    quality_score = efficiency_score = actionability_score = metrics_coverage = 0
                    
                results.append({
                    'agent_id': agent_id,
                    'scenario': scenario['scenario'],
                    'success': success,
                    'processing_time': processing_time,
                    'quality_score': quality_score,
                    'efficiency_score': efficiency_score,
                    'actionability_score': actionability_score,
                    'metrics_coverage': metrics_coverage,
                    'task_result': task_result
                })
                
            except Exception as e:
                print(f'❌ Ошибка тестирования {agent_id}: {str(e)[:60]}...')
                results.append({
                    'agent_id': agent_id,
                    'scenario': scenario['scenario'],
                    'success': False,
                    'processing_time': 0,
                    'quality_score': 0,
                    'efficiency_score': 0,
                    'actionability_score': 0,
                    'metrics_coverage': 0,
                    'error': str(e)
                })
                
        # Статистика Management уровня
        successful_tests = len([r for r in results if r['success']])
        
        if successful_tests > 0:
            avg_processing_time = sum(r['processing_time'] for r in results if r['success']) / successful_tests
            avg_quality_score = sum(r['quality_score'] for r in results if r['success']) / successful_tests
            avg_efficiency_score = sum(r['efficiency_score'] for r in results if r['success']) / successful_tests
            avg_actionability_score = sum(r['actionability_score'] for r in results if r['success']) / successful_tests
        else:
            avg_processing_time = avg_quality_score = avg_efficiency_score = avg_actionability_score = 0
            
        print(f'\n📊 СТАТИСТИКА MANAGEMENT УРОВНЯ:')
        print('=' * 40)
        print(f'✅ Успешных тестов: {successful_tests}/{len(results)}')
        print(f'⏱️ Среднее время: {avg_processing_time:.2f}с')
        print(f'📊 Среднее качество: {avg_quality_score:.1f}/100')
        print(f'⚡ Эффективность: {avg_efficiency_score:.1f}/100')
        print(f'🎯 Практичность: {avg_actionability_score:.1f}/100')
        
        return {
            'success': True,
            'level': 'management',
            'results': results,
            'stats': {
                'successful_tests': successful_tests,
                'total_tests': len(results),
                'success_rate': successful_tests / max(1, len(results)) * 100,
                'avg_processing_time': avg_processing_time,
                'avg_quality_score': avg_quality_score,
                'avg_efficiency_score': avg_efficiency_score,
                'avg_actionability_score': avg_actionability_score
            }
        }
        
    except Exception as e:
        print(f'❌ Ошибка тестирования Management агентов: {e}')
        return {
            'success': False,
            'error': str(e)
        }